In [1]:
import numpy as np
import shapely.geometry as geo
import shapely.affinity as aff

import sys
sys.path.append("../")
from electronfactors.ellipse.utilities import shapely_cutout, shapely_point

In [2]:
def test_min_distance(**kwargs):
    min_distance = kwargs['min_distance']
    point_of_interest = kwargs['point_of_interest']
    cutout = kwargs['cutout']
    
    is_within = cutout.contains(point_of_interest)
    
    distance = point_of_interest.distance(cutout.boundary)
    
    if not(is_within):
        return False
    
    elif distance < min_distance:
        return False
    
    else:
        return True

In [3]:
def furthest_possible_distance(point_of_interest, cutout):
    x = point_of_interest.xy[0][0]
    y = point_of_interest.xy[1][0]
    
    translated_cutout = aff.translate(cutout, xoff=-x, yoff=-y)
    max_bound = np.max(np.abs(translated_cutout.boundary))
    furthest_distance = max_bound * np.sqrt(2)
    
    return furthest_distance

In [4]:
def make_rays(point_of_interest, cutout, num_rays):
    x_POI = point_of_interest.xy[0][0]
    y_POI = point_of_interest.xy[1][0]
    
    ray_length = furthest_possible_distance(point_of_interest, cutout)
    theta = np.random.uniform(0, 2*np.pi, size=num_rays)
      
    x_points = x_POI + ray_length * np.cos(theta)
    y_points = y_POI + ray_length * np.sin(theta)
    
    coords = [((x_POI, y_POI), (x_points[i], y_points[i])) for i in range(num_rays)]
    rays = geo.MultiLineString(coords)
    
    return rays

In [5]:
def determine_line_segmentation(distances):
    
    begin_at_POI = np.array([distances[i][0] == 0.0 for i in range(len(distances))])
    
    segment_groups_indices  = []
    full_lines_index = np.arange(len(begin_at_POI)).astype('float')

    for i in range(1, len(begin_at_POI)):

        if (~begin_at_POI[i]) & (begin_at_POI[i-1]):
            j = int(i)
            new_segment = [j - 1]
            full_lines_index[j - 1] = np.nan

            while ~(begin_at_POI[j]):
                new_segment.append(j)
                full_lines_index[j] = np.nan
                j += 1
                if j == len(begin_at_POI):
                    break

            segment_groups_indices.append(new_segment)

    full_lines_index = full_lines_index[~np.isnan(full_lines_index)].astype('int')  

    return full_lines_index, segment_groups_indices

In [6]:
def sector_integration(num_rays=100, **kwargs):    
    x = kwargs['x']
    y = kwargs['y']
    circle_fit = kwargs['circle_fit']
    min_distance = kwargs['min_distance']
    point_of_interest = shapely_point(*kwargs['point_of_interest'])
    
    cutout = shapely_cutout(x, y)
    
    if not(test_min_distance(min_distance=min_distance, 
                             point_of_interest=point_of_interest,
                             cutout=cutout)):
        raise Exception("Point of interest is too close to edge or is outside of cutout")
    
    rays = make_rays(point_of_interest, cutout, num_rays)
    intersection = cutout.intersection(rays)
    
    line_ends = [
        geo.MultiPoint(intersection[i].coords) for i in range(len(intersection))
    ]
    
    distances = [
        [
            line_ends[j][i].distance(point_of_interest) for i in range(2)
        ] for j in range(len(intersection))
    ]
    
    full_lines_index, segment_groups_indices = determine_line_segmentation(distances)
    
    ray_factor = []
    for i in full_lines_index:
        ray_factor.append(circle_fit(distances[i][1]) - circle_fit(distances[i][0]))

    for item in segment_groups_indices:
        new_segment_factor = []
        for i in item:
            new_segment_factor.append(circle_fit(distances[i][1]) - circle_fit(distances[i][0]))

        ray_factor.append(np.sum(new_segment_factor))
    
    factor = np.mean(ray_factor)
    
    return factor
    

In [7]:
import numpy as np
from scipy.interpolate import UnivariateSpline

x = np.array([-3, -2.5, -2, 3, 3, -3, -4, -3])
y = np.array([3, -2, 3, 3, -3, -3, 3, -2])

circle_diameter = np.array([3, 4, 5, 6, 7, 8, 9])
circle_factors = np.array(
    [0.9296, 0.9562, 0.9705, 0.9858, 1.0032, 1.0067, 1.0084])


def circle_fit(radii):

    circle_radii = circle_diameter/2

    spline = UnivariateSpline(circle_radii, circle_factors)
    results = spline(radii)

    results[radii > np.max(circle_radii)] = np.max(circle_factors)
    results[radii < np.min(circle_radii)] = 0

    return results

In [8]:
test_facor = []

In [9]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [10]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [11]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [12]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [13]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [14]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [15]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [16]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [17]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [18]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [19]:
factor = sector_integration(min_distance=1.5, point_of_interest=[1,0], x=x, y=y, circle_fit=circle_fit)
test_facor.append(factor)

In [20]:
np.std(test_facor)

0.0015651931720199205